In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
from t5.data import preprocessors as prep
import functools
import t5
import gin
import sentencepiece as spm
from glob import glob
import os

gin.parse_config_file('pretrained_models_base_operative_config.gin')
vocab = 'sp10m.cased.ms-en.model'
sp = spm.SentencePieceProcessor()
sp.Load(vocab)

True

In [3]:
files = [
    '/home/husein/pure-text/dumping-parliament.txt',
    '/home/husein/pure-text/filtered-dumping-wiki.txt',
    '/home/husein/pure-text/filtered-dumping-academia.txt',
    '/home/husein/pure-text/dumping-news.txt'
]
files.extend(glob('/home/husein/pure-text/the-pile/*.txt'))

In [4]:
import re

def cleaning(string):
    string = string.replace('\n', ' ').replace('\t', ' ')
    string = re.sub(r'[ ]+', ' ', string).strip()
    return string

In [5]:
# with open(files[-2]) as fopen:
#     data = fopen.read().split('\n')
# results, result = [], []
# for i in data:
#     if not len(i) and len(result):
#         results.append(' '.join(result))
#         result = []
#     else:
#         if len(i):
#             result.append(i)
# if len(result):
#     results.append(' '.join(result))

In [6]:
for file in files:
    with open(file) as fopen:
        data = fopen.read().split('\n')
    results, result = [], []
    for i in data:
        if not len(i) and len(result):
            results.append(' '.join(result))
            result = []
        else:
            if len(i) > 10:
                result.append(i)
    if len(result):
        results.append(' '.join(result))
        
    print(file, len(results))
    s = os.path.split(file)[1]
    filename = f'{s}-pair.tsv'
    
    with tf.io.gfile.GFile(filename, 'w') as outfile:
        for i in range(len(results)):
            outfile.write('%s\t\n' % (cleaning(results[i])))

/home/husein/pure-text/dumping-parliament.txt 69487
/home/husein/pure-text/filtered-dumping-wiki.txt 363179
/home/husein/pure-text/filtered-dumping-academia.txt 951674
/home/husein/pure-text/dumping-news.txt 189786
/home/husein/pure-text/the-pile/00.jsonl-16.translated.txt 56289
/home/husein/pure-text/the-pile/00.jsonl-17.translated.txt 54941
/home/husein/pure-text/the-pile/00.jsonl-18.translated.txt 54978
/home/husein/pure-text/the-pile/00.jsonl-0.translated.txt 22498
/home/husein/pure-text/the-pile/00.jsonl-31.translated.txt 55411
/home/husein/pure-text/the-pile/00.jsonl-11.translated.txt 33000
/home/husein/pure-text/the-pile/00.jsonl-8.translated.txt 33907
/home/husein/pure-text/the-pile/00.jsonl-28.translated.txt 55562
/home/husein/pure-text/the-pile/00.jsonl-26.translated.txt 55366
/home/husein/pure-text/the-pile/00.jsonl-6.translated.txt 34305
/home/husein/pure-text/the-pile/00.jsonl-25.translated.txt 55575
/home/husein/pure-text/the-pile/00.jsonl-32.translated.txt 55309
/home/hu

In [7]:
def pair_dataset(split, shuffle_files = False):
    del shuffle_files
    ds = tf.data.TextLineDataset(
        glob('*pair.tsv')
    )

    ds = ds.map(
        functools.partial(
            tf.io.decode_csv,
            record_defaults = ['', ''],
            field_delim = '\t',
            use_quote_delim = False,
        ),
        num_parallel_calls = tf.data.experimental.AUTOTUNE,
    )
    ds = ds.map(lambda *ex: dict(zip(['text'], ex)))
    return ds


t5.data.TaskRegistry.remove('pair_dataset')
t5.data.TaskRegistry.add(
    'pair_dataset',
    dataset_fn = pair_dataset,
    splits = ['train'],
    text_preprocessor = [prep.next_sentence_prediction],
    sentencepiece_model_path = vocab,
    metric_fns = [t5.evaluation.metrics.accuracy],
)

/home/husein/.local/lib/python3.6/site-packages/t5/models/mesh_transformer.py:210: UserWarning: get_sentencepiece_model_path is deprecated. Please pass the mixture or task vocabulary directly to the Mesh TensorFlow Transformer instead.
  "get_sentencepiece_model_path is deprecated. Please pass the mixture or "


In [8]:
nq_task = t5.data.TaskRegistry.get("pair_dataset")
ds = nq_task.get_dataset(split='qa.tsv', sequence_length={"inputs": 1024, "targets": 1024})

      lambda x: x[text_key], num_parallel_calls=tf.data.experimental.AUTOTUNE)

If this is a lambda function, the error may be avoided by creating the lambda in a standalone statement.
      lambda x: x[text_key], num_parallel_calls=tf.data.experimental.AUTOTUNE)

If this is a lambda function, the error may be avoided by creating the lambda in a standalone statement.


In [9]:
r = tfds.as_numpy(ds)

In [10]:
next(r)

{'inputs_plaintext': b'nsp: Ia diperkenalkan di Colombia pada 1998, namun tidak begitu mendapat sambutan ketika itu. Daging wagyu, iaitu daging lembu Kobe berasal dari Jepun dikenali sebagai daging terbaik di dunia berdasarkan teksturnya yang lembut dan rasanya enak.',
 'inputs': array([ 2532,  7175,    50,   132,  1707,    18,  5658,    23,  9544,
          354,    29,   700,   231,  2023,   110,    24,     3, 18288,
         2522,   562,  2470,    14,    92,  2312,  3927, 20959,  1059,
           37,   437,   307,    41,  2312,   807,    18,   170,   468,
        20640,    26,    17,  6349,    16, 13318, 17382,     3,     1]),
 'targets_plaintext': b'not_next',
 'targets': array([ 699, 3835,  144, 2990,   81,    1])}